In [25]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD

# SVD Matrix Factorization

In [5]:
columns=['user_id','item_id','rating','timestamp']
frame=pd.read_csv('./ratings.csv~/ratings.csv',sep=',')
frame.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [11]:
columns=['movieId','title','release date','vid release','IMDB URL','unknown','Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir',
         'Horror','Musical','Mystery','Romance','Sci-fi','Thriller','War','Western']
movies=pd.read_csv("ml-100k/u.item",sep='|',names=columns,encoding='latin-1')
movies.head()

,movieId,title,release date,vid release,IMDB URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [12]:
combined_movies=pd.merge(frame,movies,on='movieId')
combined_movies.head()

,userId,movieId,rating,timestamp,title,release date,vid release,IMDB URL,unknown,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-fi,Thriller,War,Western
0,1,31,2.5,1260759144,Crimson Tide (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Crimson%20Tid...,0,0,...,0,0,0,0,0,0,0,1,1,0
1,7,31,3.0,851868750,Crimson Tide (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Crimson%20Tid...,0,0,...,0,0,0,0,0,0,0,1,1,0
2,31,31,4.0,1273541953,Crimson Tide (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Crimson%20Tid...,0,0,...,0,0,0,0,0,0,0,1,1,0
3,32,31,4.0,834828440,Crimson Tide (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Crimson%20Tid...,0,0,...,0,0,0,0,0,0,0,1,1,0
4,36,31,3.0,847057202,Crimson Tide (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Crimson%20Tid...,0,0,...,0,0,0,0,0,0,0,1,1,0


In [14]:
# Count No of Ratings and sort
y=combined_movies.groupby('movieId')['rating'].count().sort_values(ascending=False)
y.head()

movieId
356    341
296    324
318    311
593    304
260    291
Name: rating, dtype: int64

In [20]:
filter= combined_movies['movieId']==318
combined_movies[filter]['title'].unique()

array(["Schindler's List (1993)"], dtype=object)

# Building Utility Matrix

In [21]:
rating_cross= combined_movies.pivot_table(values='rating',index='userId',columns='title',fill_value=0)
rating_cross.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,"Wooden Man's Bride, The (Wu Kui) (1994)","Wrong Trousers, The (1993)",You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0
2,0.0,0.0,3.0,0.0,0,0.0,0.0,0.0,4,0,...,0,0.0,0.0,3.0,0.0,0.0,0,0,0.0,0
3,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0,0,3.0,0
4,0.0,0.0,0.0,0.0,0,4.0,5.0,0.0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0
5,0.0,0.0,0.0,0.0,0,0.0,4.0,0.0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0


# Transposing

In [23]:
print("Shape of Matrix :",rating_cross.shape)

Shape of Matrix : (669, 1324)


In [24]:
# USers are converted to columns and shrunk down to 12 important values 
X= rating_cross.values.T
X.shape

(1324, 669)

## Decomposing it

In [26]:
SVD =TruncatedSVD(n_components=12,random_state=17)

result=SVD.fit_transform(X)

result.shape

(1324, 12)

In [ ]:
result

# Generating Correlation matrix

In [27]:
corr_mat=np.corrcoef(result)
corr_mat.shape

(1324, 1324)

# Isolating Schindler's List

In [28]:
movies_list=list(rating_cross.columns)
schindler= movies_list.index("Schindler's List (1993)")
print(schindler)

1027


In [31]:
corr_sch= corr_mat[schindler]
corr_sch.shape

(1324,)

# Recommending the movies

In [36]:
movies_names=rating_cross.columns
list(movies_names[(corr_mat[schindler]>0.9) &  (corr_mat[schindler]<1.0)])

['Gandhi (1982)', 'Stalingrad (1993)']

In [37]:
list(movies_names[(corr_mat[schindler]>0.8) &  (corr_mat[schindler]<1.0)])

['Carmen Miranda: Bananas Is My Business (1994)',
 'City of Lost Children, The (1995)',
 'Client, The (1994)',
 'Doom Generation, The (1995)',
 'Free Willy 3: The Rescue (1997)',
 'Gandhi (1982)',
 'Heavyweights (1994)',
 "One Flew Over the Cuckoo's Nest (1975)",
 'Promesse, La (1996)',
 'Stalingrad (1993)',
 'Star Maps (1997)',
 'Star Wars (1977)',
 'Strange Days (1995)',
 'Swingers (1996)']